In [19]:
using Turing
using LimberJack
using LinearAlgebra
using CSV
using NPZ
using YAML
using ForwardDiff
using BenchmarkTools;

In [20]:
using PythonCall
sacc = pyimport("sacc")
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/jaimerz/PhD/LimberJack.jl/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

In [21]:
np = pyimport("numpy")

fol = "DESY1"
data_set = "gcgc_gcwl_wlwl"
old_meta = np.load(string("../../data/", fol, "/", data_set, "_meta.npz"))
old_files = npzread(string("../../data/", fol, "/", data_set, "_files.npz"))

sacc_path = "../../data/FD/cls_FD_covG.fits" 
yaml_path = "../../data/DESY1/gcgc_gcwl_wlwl.yml"
sacc_file = sacc.Sacc().load_fits(sacc_path)
yaml_file = YAML.load_file(yaml_path)
meta, files = make_data(sacc_file, yaml_file);

DESgc__0 DESgc__0 5
DESgc__1 DESgc__1 8
DESgc__2 DESgc__2 10
DESgc__3 DESgc__3 11
DESgc__4 DESgc__4 13
DESgc__0 DESwl__0 5
DESgc__0 DESwl__1 5
DESgc__0 DESwl__2 5
DESgc__0 DESwl__3 5
DESgc__1 DESwl__0 8
DESgc__1 DESwl__1 8
DESgc__1 DESwl__2 8
DESgc__1 DESwl__3 8
DESgc__2 DESwl__0 10
DESgc__2 DESwl__1 10
DESgc__2 DESwl__2 10
DESgc__2 DESwl__3 10
DESgc__3 DESwl__0 11
DESgc__3 DESwl__1 11
DESgc__3 DESwl__2 11
DESgc__3 DESwl__3 11
DESgc__4 DESwl__0 13
DESgc__4 DESwl__1 13
DESgc__4 DESwl__2 13
DESgc__4 DESwl__3 13
DESwl__0 DESwl__0 24
DESwl__0 DESwl__1 24
DESwl__0 DESwl__2 24
DESwl__0 DESwl__3 24
DESwl__1 DESwl__1 24
DESwl__1 DESwl__2 24
DESwl__1 DESwl__3 24
DESwl__2 DESwl__2 24
DESwl__2 DESwl__3 24
DESwl__3 DESwl__3 24


In [22]:
data_old = pyconvert(Vector{Float64}, old_meta["cls"])
inv_cov_old = Hermitian(pyconvert(Matrix{Float64}, old_meta["inv_cov"]));

In [23]:
#       Omega_b,           Omega_c          n_s,                 h,
#       DESgc__0_gc_b,     DESgc__1_gc_b,   DESgc__2_gc_b,    DESgc__3_gc_b,    DESgc__4_gc_b,
#       DESgc__0_dz,      DESgc__1_dz,      DESgc__2_dz,      DESgc__3_dz,      DESgc__4_dz,
#       wl_ia_A,          wl_ia_eta, 
#       DESwl__0_wl_m,    DESwl__1_wl_m,    DESwl__2_wl_m,    DESwl__3_wl_m,
#       DESwl__0_dz,      DESwl__1_dz,      DESwl__2_dz,      DESwl__3_dz,
#       sigma_8,          Omega_m
pars = [4.426868e-02,     2.093138e-01,     8.963611e-01,     8.495440e-01,
         1.343888e+00,    1.639047e+00,      1.597174e+00,     1.944583e+00,     2.007245e+00,
        -4.679383e-03,   -2.839996e-03,      1.771571e-03,     1.197051e-03,    -5.199799e-03,
         2.389208e-01,   -6.435288e-01, 
         1.802722e-03,   -5.508994e-03,     1.952514e-02,    -1.117726e-03,
        -1.744083e-02,    6.777779e-03,    -1.097939e-03,    -4.912315e-03,
         8.536883e-01,    2.535825e-01];

In [24]:
cls_carlos = npzread("../../chains/carlos_chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info.npz");
cls_carlos2 = npzread("../../chains/carlos_chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info_nointerp.npz");
cls_carlos3 = npzread("../../chains/carlos_chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info_nointerp_EH.npz");
cls_carlos4 = npzread("../../chains/carlos_chains/cl_cross_corr_v3_DES_2/cl_cross_corr_bestfit_info_nointerp_EH_noconv.npz");

In [25]:
npzwrite("test_cls.npz", Dict("cls"=>cls_carlos4["cls"]))

In [26]:
nuisances = Dict("DESgc__0_b" => pars[5],
                 "DESgc__1_b" => pars[6],
                 "DESgc__2_b" => pars[7],
                 "DESgc__3_b" => pars[8],
                 "DESgc__4_b" => pars[9],
                 "DESgc__0_dz" => pars[10],
                 "DESgc__1_dz" => pars[11],
                 "DESgc__2_dz" => pars[12],
                 "DESgc__3_dz" => pars[13],
                 "DESgc__4_dz" => pars[14],

                 "A_IA" => pars[15],
                 "alpha_IA" => pars[16],

                 "DESwl__0_dz" => pars[21],
                 "DESwl__1_dz" => pars[22],
                 "DESwl__2_dz" => pars[23],
                 "DESwl__3_dz" => pars[24],
                 "DESwl__0_m" => pars[17],
                 "DESwl__1_m" => pars[18],
                 "DESwl__2_m" => pars[19],
                 "DESwl__3_m" => pars[20]);

In [27]:
function make_cls(Ωm, s8; mode="EisHu", nuisances=nuisances)

    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]

    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode=mode,
                                    Pk_mode="Halofit",
                                    emul_path="../../emulator/files.npz")

    return Theory(cosmology, meta, old_files;
                  Nuisances=nuisances)

end

make_cls (generic function with 1 method)

In [37]:
using Interpolations

LoadError: UndefVarError: linear_interpolation not defined

In [30]:
cls_jaime1 = make_cls(pars[end], pars[end-1]);
cls_jaime2 = make_cls(pars[end], pars[end-1];
                      mode="emulator");

LoadError: UndefVarError: cubic_spline_interpolation not defined

In [13]:
npzwrite("jaime_cls.npz", Dict("DESY1_cls_EisHu" => cls_jaime1,
                               "DESY1_cls_emul" => cls_jaime2))

LoadError: UndefVarError: cls_jaime1 not defined

In [14]:
compa1 = @. abs(cls_carlos["cls"]-cls_jaime1)/cls_carlos["cls"];
compa12 = @. abs(cls_carlos["cls"]-cls_jaime2)/cls_carlos["cls"];
compa2 = @. abs(cls_carlos2["cls"]-cls_jaime1)/cls_carlos2["cls"];
compa3 = @. abs(cls_carlos3["cls"]-cls_jaime1)/cls_carlos3["cls"];
compa4 = @. abs(cls_carlos4["cls"]-cls_jaime1)/cls_carlos4["cls"];

println("Mean discrepancy default          ", mean(compa1))
println("Mean discrepancy default          ", mean(compa12))
println("Mean discrepancy no itp           ", mean(compa2))
println("Mean discrepancy no itp+EH        ", mean(compa3))
println("Mean discrepancy no itp+EH+noconv ", mean(compa4))

LoadError: UndefVarError: cls_jaime1 not defined

In [15]:
median(compa4)

LoadError: UndefVarError: compa4 not defined

In [16]:
chi2_carlos = cls_carlos["chi2_nolp"]
chi2_carlos2 = cls_carlos2["chi2_nolp"]
chi2_carlos3 = cls_carlos3["chi2_nolp"]
chi2_carlos4 = cls_carlos4["chi2_nolp"]
println(chi2_carlos)
println(chi2_carlos2)
println(chi2_carlos3)
println(chi2_carlos4)

495.62231927779936
495.00947244084654
495.89420547809317
497.2051355000593


In [17]:
chi2_jaime1 = dot((cls_jaime1 .- meta.data), meta.inv_cov  * (cls_jaime1 .- meta.data))

LoadError: UndefVarError: cls_jaime1 not defined

In [18]:
chi2_jaime2 = dot((cls_jaime2 .- data_old), inv_cov_old  * (cls_jaime2 .- data_old))

LoadError: UndefVarError: cls_jaime2 not defined

In [16]:
(chi2_jaime1-chi2_carlos4)/chi2_carlos4

0.030723546538246528

In [16]:
cls = [make_cls(rand(Uniform(0.2, 0.6)), rand(Uniform(0.6, 0.9))) for i in 1:1000];

In [14]:
cls_m = mean(cls)
cls_s = std(cls)
npzwrite("DES_jaime_cls.npz", Dict("cls_best" => make_cls(pars[end], pars[end-1]),
                               "cls_m" => cls_m,
                               "cls_s" => cls_s))

## plot lkl

In [13]:
#using Plots
function make_Xi(Ωm, s8;
                 pars=pars, mode="EisHu", nuisances=nuisances, 
                 data=meta.data, inv_cov=meta.inv_cov)
    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode=mode,
                                     Pk_mode="Halofit",
                                     emul_path="../../emulator/files.npz")

    theory = Theory(cosmology, meta, files;
                    Nuisances=nuisances)
    return dot((theory .- data), inv_cov  * (theory .- data))
end

make_Xi (generic function with 1 method)

In [14]:
Ωms = LinRange(0.2, 0.6, 30)
s8s = LinRange(0.6, 0.9, 30)

30-element LinRange{Float64, Int64}:
 0.6,0.610345,0.62069,0.631034,0.641379,…,0.868966,0.87931,0.889655,0.9

In [15]:
Xi2s = zeros(length(Ωms), length(s8s))
for i in 1:length(Ωms)
    for j in 1:length(s8s)
        Xi2s[i, j] = make_Xi(Ωms[i], s8s[j])
    end
end

In [16]:
Xi2s_old = zeros(length(Ωms), length(s8s))
for i in 1:length(Ωms)
    for j in 1:length(s8s)
        Xi2s_old[i, j] = make_Xi(Ωms[i], s8s[j];
                                 inv_cov=old_inv_cov)
    end
end

LoadError: UndefVarError: old_inv_cov not defined

In [17]:
npzwrite("DES_jaime_Xi2s.npz", Dict("Xi2s" =>Xi2s, "Xi2s_old" =>Xi2s_old))

# Turing model

In [25]:
@model function model(data_vector;
                      tracers_names=tracers_names,
                      pairs=pairs,
                      idx=idx,
                      cov_tot=cov_tot, 
                      files=files)
    
    #DESY1 priors
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.045)
    h  = 0.67 #~ Uniform(0.60, 0.91)
    ns = 0.96 #~ Uniform(0.87, 1.07)
    s8 = 0.811 #~ Uniform(0.6, 0.9)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = Theory(cosmology, tracers_names, pairs, idx, files)
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [26]:
iterations = 300
nadapts = 100;

In [27]:
chain3 = sample(model(data_vector), DynamicNUTS(), iterations,
                progress=true; save_state=true);

LoadError: UndefVarError: data_vector not defined

In [28]:
chain3

LoadError: UndefVarError: chain3 not defined

In [29]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

LoadError: UndefVarError: data_vector not defined